In [ ]:
import sys
import tensorflow as tf
import numpy as np
import struct


def main(argv):
    # read input arguments
    training_path = "resources/train"
    test_path = "resources/test"
    epochs = 10

    # read training data
    training_labels = read_labels(
        "{}/train-labels-idx1-ubyte".format(training_path))
    training_images = read_images(
        "{}/train-images-idx3-ubyte".format(training_path))

    # define model data
    classes_count = 10
    keep_rate = 0.8

    # define training constraints
    batch_size = 64

    # create actual neural network
    model = create_model(classes_count, keep_rate)

    # train neural network
    model.fit(training_images,
              training_labels,
              epochs=epochs,
              batch_size=batch_size)

    # load test data
    test_labels = read_labels("{}/t10k-labels-idx1-ubyte".format(test_path))
    test_images = read_images("{}/t10k-images-idx3-ubyte".format(test_path))

    # test model against test data
    result = model.evaluate(test_images, test_labels)

    print("Loss: {}\nAccuracy: {}".format(result[0], result[1]))


def create_model(classes_count, keep_rate):
    # define layers
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(1 - keep_rate),
        tf.keras.layers.Dense(classes_count, activation='softmax')
    ])

    # compile model using information about training
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


def read_labels(file_name):
    # open file
    with open(file_name, mode='rb') as file:
        file_content = file.read()

    # read number to test and amount of labels containted in this file
    [magic_number, labels_count] = struct.unpack('>ii', file_content[0:8])

    # if the magic_number does not match, something went wrong
    if magic_number != 0x0801:
        print("Magic Number does not match. (0x{:02X})".format(magic_number))
        return

    # create format string
    # this string defines the layout of file_content
    # '>' = big endian
    # 'B' = unsigned byte
    labels_format_string = '>{}B'.format(labels_count)

    # itnerpret file_content as specified by label_format_string
    labels = np.array(struct.unpack(labels_format_string, file_content[8:]))
    print("Labels loaded.")
    return labels


def read_images(file_name):
    # open file
    with open(file_name, mode='rb') as file:
        file_content = file.read()

    # load number to test against, number of images, row count and column count
    [magic_number, images_count, rows,
     cols] = struct.unpack('>iiii', file_content[0:16])

    # if magic_number does not match, something went wrong
    if magic_number != 0x0803:
        print("Magic Number does not match. (0x{:02X})".format(magic_number))
        return

    # create image array
    images = np.zeros((images_count, rows, cols), np.ubyte)
    # define format string of one image row (image in mnist files is presented row wise)
    row_format_string = '>{}B'.format(cols)
    # define start variable of actual content
    start = 16
    for image in range(0, images_count):
        for row in range(0, rows):
            end = start + cols  # define end of image row
            # load image row
            images[image, row] = np.array(
                struct.unpack(row_format_string, file_content[start:end]))
            start = end  # set start to next image

        print("Images loaded: {}".format(image + 1), end='\r')

    print()
    return images


if __name__ == '__main__':
    main(sys.argv[1:])

Labels loaded.


Images loaded: 3238

Images loaded: 15432

Images loaded: 22204

Images loaded: 25525

...

Images loaded: 41725

Images loaded: 42798

Images loaded: 47279

Images loaded: 60000


Epoch 1/10
938/938 [==============================] - 1s 1ms/step - loss: 3.0213 - accuracy: 0.7480
Epoch 2/10
802/938 [========================>.....] - ETA: 0s - loss: 0.6320 - accuracy: 0.8419